In [19]:
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import time
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains

import mapping # Package to draw the digits and use nerual networks to recognize digits

## method to save data to database

In [32]:
def html2database(html):
    
    def translate(raw):
        lst = []
        for letter in raw:
            if letter == '.':
                lst.append(letter)
            else:
                lst.append(str(mapping.mapping_num(letter, para, fontfile)))
        return ''.join(lst)

    soup = BeautifulSoup(html, "lxml")

    m_name = soup.find(class_="movie-name").string
    m_cinema = soup.title.string
    m_date = soup.find(class_="showday active").attrs['data-showday']

    woff_source = re.findall(r"data:application/font-woff;charset=utf-8;base64,(.\S+)[)] format", html)[0]

    para = mapping.load_para()
    fontfile = mapping.make_woff(woff_source)

    info_lst =[]
    if soup.tbody == None:
        pass
    else:
        for tag in soup.tbody.children:
            m_time = tag.strong.string
            m_type = tag.find(class_ ="stl-ver").string
            m_price_raw = tag.find(class_="stonefont").string

            m_price = translate(m_price_raw)

            info_lst.append((m_time, m_type, m_price))

        import dateutil.parser
        import sqlite3

        dateutil.parser.parse("2017-12-25 09:25")

        info_lst

        con = sqlite3.connect("data.sqlite")
        cur = con.cursor()

        cur.execute('''CREATE TABLE IF NOT EXISTS Moive
                    (id INTEGER UNIQUE, cinema TEXT, name TEXT, time DATETIME, type TEXT, price DOUBLE)''')

        cur.execute('''SELECT MAX(id) FROM Moive''')
        num = cur.fetchone()

        if num[0] == None:
            num =0
        else:
            num = num[0]

        for m_id, info in enumerate(info_lst):
            cur.execute('''INSERT INTO Moive(id, cinema, name, time, type, price)
                        VALUES (?, ?, ?, ?, ?, ?)''', 
                        (num+m_id+1, m_cinema, m_name, dateutil.parser.parse(m_date+' '+info[0]), info[1], info[2]))

        con.commit()    

## Scraping from the given url

In [34]:
driver = webdriver.Chrome()
driver.get("http://m.maoyan.com/shows/882?_v_=yes&date=2017-01-01")

elements = driver.find_elements_by_xpath("//img")
for ele in elements:
    ele.click()
    html2database(driver.page_source)
    time_elements = driver.find_elements_by_xpath("//li[@class='showday']")
    for time_element in time_elements:
        time_element.click()
        html2database(driver.page_source)